**Universidad del Valle de Guatemala** <br>
**Computación Paralela** <br>
**Corto 15** <br>
**Andres Quezada 21085**

In [1]:
import torch
torch.cuda.is_available()


False

In [20]:
%%writefile reduceSum.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>

// Kernel CUDA para reducción paralela optimizada
__global__ void reduceSum(int *input, int *output, int n) {
    extern __shared__ int sharedData[];

    unsigned int tid = threadIdx.x;
    unsigned int idx = blockIdx.x * blockDim.x * 2 + threadIdx.x;

    // Cargar elementos en memoria compartida para utilizar la coalescencia de memoria
    int sum = 0;
    if (idx < n)
        sum = input[idx];
    if (idx + blockDim.x < n)
        sum += input[idx + blockDim.x];
    sharedData[tid] = sum;
    __syncthreads();

    // Realizar la reducción en memoria compartida
    for (unsigned int s = blockDim.x / 2; s > 32; s >>= 1) {
        if (tid < s) {
            sharedData[tid] += sharedData[tid + s];
        }
        __syncthreads();
    }

    // Desenrollar el último warp para evitar __syncthreads()
    if (tid < 32) {
        volatile int *vsmem = sharedData;
        vsmem[tid] += vsmem[tid + 32];
        vsmem[tid] += vsmem[tid + 16];
        vsmem[tid] += vsmem[tid + 8];
        vsmem[tid] += vsmem[tid + 4];
        vsmem[tid] += vsmem[tid + 2];
        vsmem[tid] += vsmem[tid + 1];
    }

    // Escribir el resultado de este bloque en memoria global
    if (tid == 0)
        output[blockIdx.x] = sharedData[0];
}

int main(int argc, char **argv) {
    // Verificar si se proporcionó el argumento del exponente
    if (argc != 2) {
        fprintf(stderr, "Usage: %s <exponent>\n", argv[0]);
        fprintf(stderr, "Example: %s 24 (for N = 1 << 24)\n", argv[0]);
        return -1;
    }

    // Convertir el argumento a un entero
    int exponent = atoi(argv[1]);
    if (exponent < 0 || exponent > 30) {
        fprintf(stderr, "Exponent must be between 0 and 30.\n");
        return -1;
    }

    int N = 1 << exponent;
    size_t size = N * sizeof(int);

    // Asignar e inicializar memoria en el host
    int *h_input = (int *)malloc(size);
    if (h_input == NULL) {
        fprintf(stderr, "Failed to allocate host vectors!\n");
        return -1;
    }
    for (int i = 0; i < N; i++) {
        h_input[i] = rand() % 100; // Valores aleatorios entre 0 y 99
    }

    // Asignar memoria en el dispositivo
    int *d_input, *d_output;
    cudaMalloc((void **)&d_input, size);

    // Crear eventos CUDA para medir el tiempo
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Iniciar medición de tiempo desde la transferencia de datos CPU -> GPU
    cudaEventRecord(start, 0);

    // Copiar datos desde el host al dispositivo
    cudaMemcpy(d_input, h_input, size, cudaMemcpyHostToDevice);

    // Determinar tamaños de bloque y grid
    int threadsPerBlock = 256;
    int blocksPerGrid = (N + threadsPerBlock * 2 - 1) / (threadsPerBlock * 2);

    // Asignar arreglo de salida en el dispositivo
    int *h_partialSums = (int *)malloc(blocksPerGrid * sizeof(int));
    cudaMalloc((void **)&d_output, blocksPerGrid * sizeof(int));

    // Lanzar el kernel
    reduceSum<<<blocksPerGrid, threadsPerBlock, threadsPerBlock * sizeof(int)>>>(d_input, d_output, N);

    // Copiar las sumas parciales de vuelta al host
    cudaMemcpy(h_partialSums, d_output, blocksPerGrid * sizeof(int), cudaMemcpyDeviceToHost);

    // Detener la medición de tiempo después de la transferencia de datos GPU -> CPU
    cudaEventRecord(stop, 0);
    cudaEventSynchronize(stop);

    float elapsedTime;
    cudaEventElapsedTime(&elapsedTime, start, stop);

    // Reducción final en el host
    int gpu_sum = 0;
    for (int i = 0; i < blocksPerGrid; i++) {
        gpu_sum += h_partialSums[i];
    }

    // Validar el resultado realizando la suma en la CPU
    // Medir el tiempo de la CPU
    clock_t cpu_start = clock();
    int cpu_sum = 0;
    for (int i = 0; i < N; i++) {
        cpu_sum += h_input[i];
    }
    clock_t cpu_end = clock();
    float cpu_time = 1000.0 * (cpu_end - cpu_start) / CLOCKS_PER_SEC;

    printf("Array Size (N): %d\n", N);
    printf("CPU Sum: %d\n", cpu_sum);
    printf("GPU Sum: %d\n", gpu_sum);
    printf("Difference: %d\n", abs(cpu_sum - gpu_sum));
    printf("GPU Time (including data transfer): %f ms\n", elapsedTime);
    printf("CPU Time: %f ms\n", cpu_time);

    // Liberar memoria
    free(h_input);
    free(h_partialSums);
    cudaFree(d_input);
    cudaFree(d_output);

    cudaEventDestroy(start);
    cudaEventDestroy(stop);

    return 0;
}


Overwriting reduceSum.cu


In [21]:
!nvcc -o reduceSum reduceSum.cu


In [22]:
!./reduceSum 24


Array Size (N): 16777216
CPU Sum: 830584179
GPU Sum: 830584179
Difference: 0
GPU Time (including data transfer): 15.253568 ms
CPU Time: 43.313999 ms
